# Problem Sheet 8 - Tensors and Tensor Decompositions

In this exercise we consider tensors and tensor decompositions. We work with tensorly and parts of this notebook have been adapted from https://github.com/JeanKossaifi/tensorly-notebooks

In [3]:
pip install tensorly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 5.6 MB/s eta 0:00:00


In [4]:
import numpy as np
import tensorly as tl

# 1. Creating a tensor

A tensor can be represented in multiple ways. The simplest is the slice representation through multiple matrices.

Let's take for this example the tensor $\tilde X$ defined by its frontal slices:

$$
   X_1 = 
   \left[
   \begin{matrix}
   0  & 2  & 4  & 6\\
   8  & 10 & 12 & 14\\
   16 & 18 & 20 & 22
   \end{matrix}
   \right]
$$

and 

$$
   X_2 =
   \left[
   \begin{matrix}
   1  & 3  & 5  & 7\\
   9  & 11 & 13 & 15\\
   17 & 19 & 21 & 23
   \end{matrix}
   \right]
$$


In [5]:
print('In Python, this array can be expressed as a numpy array and converted to a tensorly tensor:')

X = tl.tensor(np.arange(24).reshape((3, 4, 2)), dtype=tl.float64)
print(X)

In Python, this array can be expressed as a numpy array and converted to a tensorly tensor:
[[[ 0.  1.]
  [ 2.  3.]
  [ 4.  5.]
  [ 6.  7.]]

 [[ 8.  9.]
  [10. 11.]
  [12. 13.]
  [14. 15.]]

 [[16. 17.]
  [18. 19.]
  [20. 21.]
  [22. 23.]]]


**Task:** Recall the definitions of fibers and slices and look at the tensor through them.

# 2. Basic tensor operations
## 2.1 Matrization/Unfolding

You have learned about matrization/unfolding in the lecture.
It turns out that there's an alternative (computationally more efficient) convention to do this that is implemented in the python library ``tensorly``.

**Task:** Read about the two different conventions here: 

http://jeankossaifi.com/blog/unfolding.html

The definition you know from the lecture is referred to as "Kolda".
Consider the tensor $\tilde X$ previously defined by
$$
   X_1 = 
   \left[
   \begin{matrix}
   0  & 2  & 4  & 6\\
   8  & 10 & 12 & 14\\
   16 & 18 & 20 & 22
   \end{matrix}
   \right]
$$

and 

$$
   X_2 =
   \left[
   \begin{matrix}
   1  & 3  & 5  & 7\\
   9  & 11 & 13 & 15\\
   17 & 19 & 21 & 23
   \end{matrix}
   \right]
$$

A possible implementation of the "Kolda" unfolding is given here:

In [6]:
def reorder(indices, mode):
    """Reorders the elements
    """
    indices = list(indices)
    element = indices.pop(mode)
    return ([element] + indices[::-1])

def unfold_Kolda(tensor, mode):
    """Returns the mode-`mode` unfolding of `tensor`
    """
    return np.transpose(tensor, reorder(range(tensor.ndim), mode)).reshape((tensor.shape[mode], -1))

Play with unfolding $\tilde X$ using the two different conventions.

In [7]:
unfold_Kolda(X, mode=0)

array([[ 0.,  2.,  4.,  6.,  1.,  3.,  5.,  7.],
       [ 8., 10., 12., 14.,  9., 11., 13., 15.],
       [16., 18., 20., 22., 17., 19., 21., 23.]])

In [8]:
tl.unfold(X, mode=0)

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11., 12., 13., 14., 15.],
       [16., 17., 18., 19., 20., 21., 22., 23.]])

## 2.2 Folding

Folding is the inverse operation: you can **fold** an unfolded tensor back from the matrix to a full tensor.
Of course, the two conventions have to be implemented differently.
You can use either the ``tensorly.fold`` function or the "Kolda" version given below.

**Task:** Convince yourself that folding the unfolded tensor gives back the original tensor for both conventions.

In [9]:
def fold_Kolda(unfolded, mode, shape):
    """Returns the folded tensor of shape `shape` from the `mode`-mode unfolding `unfolded`.
    """
    unfolded_indices = reorder(range(len(shape)), mode)
    original_shape = [shape[i] for i in unfolded_indices]
    unfolded = unfolded.reshape(original_shape)

    folded_indices = list(range(len(shape)-1, 0, -1))
    folded_indices.insert(mode, 0)
    return np.transpose(unfolded, folded_indices)

In [11]:
print('X:')
print(X)
original_shape = X.shape

mode = 1
unfolded = tl.unfold(X, mode)
print('unfolded:')
print(unfolded)

folded = tl.fold(unfolded, mode, original_shape)
print('folded again:')
print(folded)

X:
[[[ 0.  1.]
  [ 2.  3.]
  [ 4.  5.]
  [ 6.  7.]]

 [[ 8.  9.]
  [10. 11.]
  [12. 13.]
  [14. 15.]]

 [[16. 17.]
  [18. 19.]
  [20. 21.]
  [22. 23.]]]
unfolded:
[[ 0.  1.  8.  9. 16. 17.]
 [ 2.  3. 10. 11. 18. 19.]
 [ 4.  5. 12. 13. 20. 21.]
 [ 6.  7. 14. 15. 22. 23.]]
folded again:
[[[ 0.  1.]
  [ 2.  3.]
  [ 4.  5.]
  [ 6.  7.]]

 [[ 8.  9.]
  [10. 11.]
  [12. 13.]
  [14. 15.]]

 [[16. 17.]
  [18. 19.]
  [20. 21.]
  [22. 23.]]]


## 2.3 n-mode product

Good new: in this subsection, we don't have to worry about different conventions anymore! So let's use what ``TensorLy`` has in store.

In TensorLy, all the tensor algebra functions are located in the `tensorly.tenalg` module. For the n-mode product, you will need to use the function `mode_dot` that works transparently for multiplying a tensor by a matrix or a vector along a given mode.

#### Tensor times matrix

With the tensor $\tilde X$ of size (3, 4, 2) we defined previously, let's define a matrix M of size (5, 4) to multiply along the second mode:

In [12]:
M = tl.tensor(np.arange(4*5).reshape((5, 4)))
print('M:')
print(M)
result = tl.tenalg.mode_dot(X, M, mode=1)
print('2-mode product of X and M, result:')
print(result)

M:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]]
2-mode product of X and M, result:
[[[  28.   34.]
  [  76.   98.]
  [ 124.  162.]
  [ 172.  226.]
  [ 220.  290.]]

 [[  76.   82.]
  [ 252.  274.]
  [ 428.  466.]
  [ 604.  658.]
  [ 780.  850.]]

 [[ 124.  130.]
  [ 428.  450.]
  [ 732.  770.]
  [1036. 1090.]
  [1340. 1410.]]]


#### Tensor times vector

Similarly, we can contract along the second mode with a vector of size 4 (our tensor is of size (3, 4, 2)).

In [13]:
v = tl.tensor(np.arange(4))
print('v:', v)
result = tl.tenalg.mode_dot(X, v, mode=1)
print('result:')
print(result)

v: [0 1 2 3]
result:
[[ 28.  34.]
 [ 76.  82.]
 [124. 130.]]


In [14]:
print('Here, we could form each column (fiber) of the result, by taking the dot product between the frontal slices of X and the vector:')
print(X[:, :, 0] @ v)
print(X[:, :, 1] @ v)
print('And we can form the whole product by concanating these as the column of the resulting matrix:')
np.stack([X[:, :, 0] @ v, X[:, :, 1] @ v]).T

Here, we could form each column (fiber) of the result, by taking the dot product between the frontal slices of X and the vector:
[ 28.  76. 124.]
[ 34.  82. 130.]
And we can form the whole product by concanating these as the column of the resulting matrix:


array([[ 28.,  34.],
       [ 76.,  82.],
       [124., 130.]])

In [15]:
print('We could equivalently use the mode-0 slices:')
for i in range(3):
    print(X[i, ...].T @ v)
print('And we can form the whole result by stacking these as the rows of the matrix:')
np.vstack([X[i, ...].T @ v for i in range(3)])

We could equivalently use the mode-0 slices:
[28. 34.]
[76. 82.]
[124. 130.]
And we can form the whole result by stacking these as the rows of the matrix:


array([[ 28.,  34.],
       [ 76.,  82.],
       [124., 130.]])

# 3. Khatri-Rao product

Recall the Khatri-Rao product $A \odot B$ of the two matrices $A \in \mathbb{R}^{I_1,J}$ and $B \in \mathbb{R}^{I_2,J}$. 

**Task:** Write a function that takes two matrices with the same number of columns as an input and returns their Khatri-Rao product.

In [16]:
def my_khatri_rao(A, B):
    # Get the number of columns in A and B
    num_cols = A.shape[1]

    # Initialize an empty matrix for the Khatri-Rao product
    R = np.empty((A.shape[0] * B.shape[0], num_cols))

    # Compute the Khatri-Rao product column-wise
    for col in range(num_cols):
        R[:, col] = np.kron(A[:, col], B[:, col])

    return R

Compare the output of your function with that of TensorLy: ``tl.tenalg.khatri_rao`` to check if it works.

In [17]:
A = np.reshape(np.arange(1,10), (3,3))
B = np.reshape(np.arange(1,13), (4,3))
tl.tenalg.khatri_rao((A, B))

array([[  1,   4,   9],
       [  4,  10,  18],
       [  7,  16,  27],
       [ 10,  22,  36],
       [  4,  10,  18],
       [ 16,  25,  36],
       [ 28,  40,  54],
       [ 40,  55,  72],
       [  7,  16,  27],
       [ 28,  40,  54],
       [ 49,  64,  81],
       [ 70,  88, 108]])

# 4. Canonical Polyadic (CP) decomposition

In the lecture, you learned about the CP (Candecomp, Parafac/Canonical Polyadic) decomposition, which returns a so-called Kruskal form of a tensor
$$
\tilde X = \sum_{r=1}^R \lambda_r a_r^{(1)} \circ a_r^{(2)} \circ a_r^{(3)}.
$$
Computing the rank $R$ of a tensor is an NP-hard problem, but we can use the CP decomposition as an approximation when choosing a rank for the result. 
It can be computed by an Alternating Least Squares (ALS) scheme.

**Task:** Complete the code below, which computes the CP using ALS for a tensor with 3 modes.

**Hint:** You can find two ways to do this in the lecture notes.

In [18]:
def my_cp_decomposition_als(tensor_X, rank_R, num_iter):
    #initialize A1, A2, A3 using left leading singular vectors
    X1 = tl.unfold(tensor_X, mode=0)
    A1 = np.linalg.svd(X1)[0][:,:rank_R]
    X2 = tl.unfold(tensor_X, mode=1)
    A2 = np.linalg.svd(X2)[0][:,:rank_R]
    X3 = tl.unfold(tensor_X, mode=2)
    A3 = np.linalg.svd(X3)[0][:,:rank_R]
    
    for iter in range(num_iter):
        A1 = A1 # CHANGE!
        lambda_array = np.linalg.norm(A1, axis=0)
        for i in range(rank_R):
            A1[:,i] = A1[:,i] / lambda_array[i]
        A2 = A2 # CHANGE!
        lambda_array = np.linalg.norm(A2, axis=0)
        for i in range(rank_R):
            A2[:,i] = A2[:,i] / lambda_array[i]
        A3 = A3 # CHANGE!
        lambda_array = np.linalg.norm(A3, axis=0)
        for i in range(rank_R):
            A3[:,i] = A3[:,i] / lambda_array[i]
    return lambda_array, (A1,A2,A3)

In [19]:
print('X:', X)
ww, ff = my_cp_decomposition_als(X, 2, 20)
full_tensor = tl.kruskal_to_tensor((ww, ff))
print('\nReconstruction from CP decomposition:')
print(full_tensor)

X: [[[ 0.  1.]
  [ 2.  3.]
  [ 4.  5.]
  [ 6.  7.]]

 [[ 8.  9.]
  [10. 11.]
  [12. 13.]
  [14. 15.]]

 [[16. 17.]
  [18. 19.]
  [20. 21.]
  [22. 23.]]]

Reconstruction from CP decomposition:
[[[ 0.43796008 -0.50009678]
  [ 0.14086623 -0.23613345]
  [-0.15622763  0.02782987]
  [-0.45332148  0.2917932 ]]

 [[ 0.01261587 -0.28672318]
  [-0.09994788 -0.2264667 ]
  [-0.21251164 -0.16621022]
  [-0.3250754  -0.10595374]]

 [[-0.41272833 -0.07334958]
  [-0.34076199 -0.21679995]
  [-0.26879565 -0.36025032]
  [-0.19682931 -0.50370068]]]


Compare the results of your implementation with Tensorly.

In [20]:
from tensorly.decomposition import parafac
weights, factors = parafac(X, rank=2)
full_tensor = tl.kruskal_to_tensor((weights, factors))
print(full_tensor)

[[[-0.05785963  1.06474682]
  [ 1.99072479  3.01009381]
  [ 4.03930922  4.95544081]
  [ 6.08789364  6.9007878 ]]

 [[ 8.02013459  8.98316657]
  [10.01080986 10.99006258]
  [12.00148513 12.9969586 ]
  [13.9921604  15.00385461]]

 [[16.09812881 16.90158632]
  [18.03089492 18.97003135]
  [19.96366103 21.03847639]
  [21.89642715 23.10692142]]]


# Additional task: Tucker Decomposition a.k.a. HOSVD

**Task:** Implement the higher-order SVD (HOSVD) for 3-mode tensors.

In [26]:
def my_hosvd(tensor_X, ranks):
    unfolded_modes = [tl.unfold(tensor_X, mode) for mode in range(3)]

    # Compute the SVD for each unfolded mode
    factor_matrices = [np.linalg.svd(unfolded, full_matrices=False)[0] for unfolded in unfolded_modes]

    # Reconstruct the core tensor
    core_tensor = tl.tenalg.multi_mode_dot(tensor_X, factor_matrices, transpose=True)

    # Extract the factor matrices
    A1, A2, A3 = factor_matrices

    return core_tensor, A1, A2, A3


If implemented correctly, you should be able to execute the following code:

In [27]:
G,A1,A2,A3 = my_hosvd(X, [3, 4, 2])
print('Core tensor G:')
print(G)
full_tensor = tl.tucker_to_tensor((G, [A1,A2,A3]))
print('\nFull tensor reconstructed from HOSVD:')
print(full_tensor)

Core tensor G:
[[[-6.55274958e+01  2.81237824e-02]
  [ 6.49654739e-03  3.14031179e-01]
  [ 6.16146308e-15  1.04726206e-16]
  [-5.54896385e-15  1.79850393e-15]]

 [[-2.68022035e-03 -1.17061732e+00]
  [-5.34408458e+00 -3.43876182e-01]
  [ 2.14368277e-15  1.79530866e-16]
  [-1.31301759e-16 -1.30213746e-16]]

 [[-5.13731398e-15  1.17272774e-15]
  [ 5.47004159e-15  7.98600625e-16]
  [ 3.46382049e-16 -5.29650556e-17]
  [ 3.66182703e-16  9.62504943e-16]]]

Full tensor reconstructed from HOSVD:
[[[4.69555017e-15 1.00000000e+00]
  [2.00000000e+00 3.00000000e+00]
  [4.00000000e+00 5.00000000e+00]
  [6.00000000e+00 7.00000000e+00]]

 [[8.00000000e+00 9.00000000e+00]
  [1.00000000e+01 1.10000000e+01]
  [1.20000000e+01 1.30000000e+01]
  [1.40000000e+01 1.50000000e+01]]

 [[1.60000000e+01 1.70000000e+01]
  [1.80000000e+01 1.90000000e+01]
  [2.00000000e+01 2.10000000e+01]
  [2.20000000e+01 2.30000000e+01]]]


Compare your results with Tensorly.

In [28]:
from tensorly.decomposition import tucker
core, factors = tucker(X, rank=[3, 4, 2])
print('Core tensor G:')
print(core)
full_tensor = tl.tucker_to_tensor((core, factors))
print('\nFull tensor reconstructed from HOSVD:')
print(full_tensor)

Core tensor G:
[[[ 6.55274958e+01 -2.81237824e-02]
  [-6.49654739e-03 -3.14031179e-01]
  [ 8.44200415e-15  3.64395164e-16]
  [-4.34527528e-15  1.26611803e-15]]

 [[-2.68022035e-03 -1.17061732e+00]
  [-5.34408458e+00 -3.43876182e-01]
  [-9.22296351e-16 -9.82315860e-17]
  [-9.71645063e-16  6.31177723e-16]]

 [[ 5.57431378e-16 -1.18877633e-15]
  [-2.97724080e-15  1.05265532e-17]
  [-2.54129728e-16 -3.25113537e-16]
  [ 6.97897582e-17  7.83396173e-16]]]

Full tensor reconstructed from HOSVD:
[[[-4.50674066e-15  1.00000000e+00]
  [ 2.00000000e+00  3.00000000e+00]
  [ 4.00000000e+00  5.00000000e+00]
  [ 6.00000000e+00  7.00000000e+00]]

 [[ 8.00000000e+00  9.00000000e+00]
  [ 1.00000000e+01  1.10000000e+01]
  [ 1.20000000e+01  1.30000000e+01]
  [ 1.40000000e+01  1.50000000e+01]]

 [[ 1.60000000e+01  1.70000000e+01]
  [ 1.80000000e+01  1.90000000e+01]
  [ 2.00000000e+01  2.10000000e+01]
  [ 2.20000000e+01  2.30000000e+01]]]
